## Ingestion to Azure Cache for Redis

### Set environment variables

In [ ]:
## TODO

#### Create table

In [ ]:
## TODO

#### Ingest text sample with embeddings

In [ ]:
## TODO

#### Ingest doc sample with embeddings

In [ ]:
## TODO

#### Ingest image sample with embeddings

In [ ]:
## TODO